# Patterns in Cell Cycle Gene Expression 

The project is an example of "omics" approaches that measure everything in a population - genomics for genes, transcriptomics for RNA transcripts, proteomics for proteins and so on. Big data like this are becoming more and more popular in biology as they are information-rich and relatively cheap. They require a lot of data exploration and organisation, and are even more useful when many big datasets can be compared at once.

> "The dream of every cell is to become two cells." - Francois Jacob

Cells grow and divide, in the process replicating their DNA, segregating the resulting chromosomes into 2 daughter cells, along with replicating every other part of the cell and checking that all is well at every step. This "cell cycle" involves co-ordinated processes including "gene expression" - transcribing new RNA, translating new proteins to supply the machinery for each stage, and degrading these when they're not needed.

One route to understanding the cell cycle is to measure everything that's going on in cells, and exploring the data to extract patterns. This project uses a dataset of cell-cycle-dependent gene expression in budding yeast *Saccharomyces cerevisiae*, a unicellular eukaryote that is a useful model organism for understanding shared biological processes.

The data are taken from the paper:

> Investigating Conservation of the Cell-Cycle-Regulated Transcriptional Program in the Fungal Pathogen, *Cryptococcus neoformans*.
> Christina M. Kelliher, Adam R. Leman, Crystal S. Sierra, Steven B. Haase. PLoS Genetics, 2016. https://doi.org/10.1371/journal.pgen.1006453

The paper compared datasets from 2 different species of yeast. Here we will focus only on one of them, *Saccharomyces cerevisiae*.


# Yeast gene expression dataset

The *S. cerevisiae*. dataset was generated by first arresting the cell cycle in a culture of yeast cells using a yeast pheromone called alpha factor. Then, the pheromone was washed out, which allows the culture to start the cell cycle synchronously. Samples were taken every 10 minutes and RNA extracted, sequenced, and analysed, giving an RNA-seq or trancriptomic dataset. The dataset gives estimates of the relative amount of mRNA expressed from every yeast gene in the cell across several successive cell cycles.

These data are in the file `GSE80474_Scerevisiae_normalized.txt`, which was part of the supplementary data for the paper.


## Setting up the analysis environment

First we need to load our analysis modules.

In [ ]:
# Analysis modules
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels.api as sm
np.set_printoptions(precision=5, suppress=True)  # suppress scientific floatation 
sns.set(color_codes=True)
%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Read in the data

Let's read the data into the python session and look at it.

In [ ]:
df = pd.read_table("GSE80474_Scerevisiae_normalized.txt")

In [ ]:
df.head()

In [ ]:
df.tail()

## What is in the data, really?

This is a real-life dataset taken from a real-life published paper.
We will have to think about what the data mean. 

To get started, it's always a good idea to read the paper that the data came from. The main part of the paper says:

> Total RNA was extracted from yeast cells at each time point (every 5 minutes for S. cerevisiae [...]) and multiplexed for stranded RNA-Sequencing.

The supplementary methods of the paper (Supplementary data S1) give more details on how the data were analysed:

> Transcript quantification of annotated yeast genes was performed using alignment files output from STAR and Cufflinks2 [4]. Time point samples from the respective yeasts were then normalized together using the CuffNorm feature. The normalized FPKM gene expression outputs (“genes.fpkm_table”) were used in the analyses presented. To avoid fractional and zero values, 1 was added to every FPKM value in each dataset using the R statistical programming environment [5]. Fractions and zeros were found to interfere with the periodicity algorithms that involved log-transformation of data points (data not shown). Normalized gene expression data for each yeast are available in S1 and S2 Tables. Raw RNA-Sequencing data from this manuscript have been submitted to the NCBI Gene Expression Omnibus (GEO; http://www.ncbi.nlm.nih.gov/geo/) under accession number GSE80474.

This probably makes little sense if it's your first time seeing RNA-seq.

If you want to know more about the background and how RNA-seq works, the RNAseqlopedia is a great resource: https://rnaseq.uoregon.edu. 


## What you need to know about the data

For this project you need to know:

- Each column is a time point
- Except the first column, that contains gene names or identifiers (more about those below)
- Each row is a gene
- The numbers are estimates of how much of each mRNA is present in each time point
- Precisely, the estimates are in units are Fragments per Kilobase per Million reads (FPKM)
- The authors added 1 every FPKM value to avoid later problems from taking log transformations of zero

Because of this, we will rename the first column of the data frame to `gene`, which is a better description of the contents of the column.

In [ ]:
df = df.rename(columns = {'time_points':'gene'})

## Why are they talking about log transformation?

Log transformation is important in many kinds of data analysis and quantitative thinking. Log transformation turns multiplication and division (which are hard) into addition and subtraction (which are easier). Log transformation is also useful for displaying data, where if some of your data values are 100x as large as your other values, log transformations can make them easier to visualise. [This YouTube video by Rafael Irizarry explains log transformation in biology](https://www.youtube.com/watch?v=3huF0DwxCtU).

To see this, let's take a look at the distribution of some of the data.

To get a feel for it, we can focus on a single timepoint at 0 minutes.

In [ ]:
estimates_0min = df["0"]
estimates_0min

Plotting a histogram of the distribution in the native scale puts many values close to zero, and doesn't look like anything much:

In [ ]:
sns.histplot(x = estimates_0min)

Plotting a histogram of the distribution on a log10-scale is more helpful:

In [ ]:
sns.histplot(x = np.log10(estimates_0min))

Here we see that there are many values close to zero, that's the tall bar on the left.
Then there's a broad distribution of expression values over many orders of magnitude.
That means that there are some mRNAs that are 10000 times as abundant as other mRNAs.

You could check if these patterns hold at other timepoints too.

The take-home message: **if your plot doesn't look good or your results don't make sense, try a log-transformation**.

## Tidying the data

These data are not tidy...

> In tidy data: Every column is a variable. Every row is an observation.

Tidying the data will make it easier to plot.

My goal here is that each observation is: one gene, one timepoint, one mRNA abundance. I'm going to call the mRNA abundance estimate FPKM1 to remind me that it is in units of FPKM plus one.

So I'll use `df.melt` to make a data frame with columns `gene`, `time_point`, `FPKM1`.
We also need to make sure that `time_point` is recorded as a number.

In [ ]:
df_tidy = df.melt(id_vars = "gene", var_name = "time_point", value_name = "FPKM1")
df_tidy["time_point"] = pd.to_numeric(df_tidy["time_point"], downcast="integer")
df_tidy.head()

In [ ]:
# check the data types
df_tidy.dtypes

## Plot abundance for a gene

Some genes called "cyclins" help to regulate the cell cycle. In yeast these genes are sensibly called CLN1, CLN2, CLN3. There are also B-type cyclins called CLB1 through CLB6.

Let's visualise the mRNA abundance for some of these genes across the cell cycle, starting with cyclin 1.

In [ ]:
df_tidy_CLN1 = df_tidy[df_tidy.gene == "CLN1"]
df_tidy_CLN1

In [ ]:
sns.lineplot(data = df_tidy_CLN1, x="time_point", y="FPKM1")

In [ ]:
sns.lineplot(data = df_tidy[df_tidy.gene == "CLN2"], x="time_point", y="FPKM1")

Instead of plotting each gene independently, we can plot multiple genes on the same axes in different colours to distinguish them.

In [ ]:
df_tidy_CLN123 = df_tidy[df_tidy.gene.isin(("CLN1", "CLN2", "CLN3"))]

sns.lineplot(data = df_tidy_CLN123, x="time_point", y="FPKM1", hue = "gene")

What if we include both the cyclins and the B-type cyclins?

In [ ]:
df_tidy_CLNCLB = df_tidy[df_tidy.gene.isin(("CLN1", "CLN2", "CLN3", "CLB1", "CLB2", "CLB3", "CLB4", "CLB5", "CLB6"))]

sns.lineplot(data = df_tidy_CLNCLB, x="time_point", y="FPKM1", hue = "gene")

That lineplot is incomprehensible, there's too many overlapping colours. We'll have to try a different approach. Let's make a heatmap.

In [ ]:
df_tidy_CLNCLB_pivot = df_tidy_CLNCLB.pivot(columns = "time_point", index = "gene")

sns.heatmap(data = df_tidy_CLNCLB_pivot)

In this heatmap it's reasonably clear that CLN3 is expressed first. Then CLN2, CLN1, CLB6 at about the same time. Then CLB1 and CLB2 later. But CLB3 and CLB4 are barely visible because they have lower peak abundance.

So let's try to plot on a log scale.

In [ ]:
sns.heatmap(data = np.log10(df_tidy_CLNCLB_pivot))

That doesn't look better. Let's try once more changing the colourmap to a "diverging" colourmap so we can see differences in a different way. 

[Help page on matplotlib colourmaps](https://matplotlib.org/stable/users/explain/colors/colormaps.html).

In [ ]:
sns.heatmap(data = np.log10(df_tidy_CLNCLB_pivot), cmap="bwr")

This heatmap makes the differences a bit clearer.

What else could you do to make the meaning of the figure more clear? What labels and information are needed to tell a story? For a start, which x-axis and y-axis labels would help?

## Log fold-change

One problem with these plots is that they show abundance of each mRNA relevant to other mRNAs.

What if we need to compare *relative* abundance - i.e. the change in abundance of an mRNA relative to its average? This is an example of scaling and normalisation.

A standard and sensible approach is log fold-change:

- calculate the log of each FPKM1
- calculate the mean log value separately for each gene
- the log fold-change is the log value minus the mean value, separately for each gene.

You should be able to calculate the log fold-change in a new data frame using tools you've learned on the course, including [pandas.DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html). Mostly in genomics people would use a log2 fold-change because it's easy for humans to interpret 2-fold up or 2-fold down. If you need to remind yourself what log2 (logarithm base 2) is, go ahead, it's important.

Then it's helpful to use a heatmap with a diverging colourmap like `bwr`, and with the `center` value set to 0, meaning that when log2 fold change is 0 the gene has its average value. Why not try it?


## What do these gene expression plots over time mean?

These plots show that different cyclins have high expression at different stages of the cell cycle.

Compare that with [Figure 1 of the paper that measures the proportion of cells budded at each timepoint](https://doi.org/10.1371/journal.pgen.1006453.g001).

Can you see how the up-and down correspond to the population of cells starting synchronously, and then becoming less synchronous? So that the time of first budding is about 60 minutes in? Then about 140 minutes cells start to produce second buds, but the peak is broader and wider so the times at which budding happens are spread out over the population?

This also shows in the gene expression. Cell-cycle dependent genes have tight tall peaks in expression during the first cell cycle, then broader and wider peaks at the cell cycle, and at later cycles the cells are no longer synchronised so the peak is barely visible.

When is each cyclin expressed relative to the emergence of buds?

Several research project questions could go along similar lines: which other genes are expressed at which point of the cell cycle? Which functions are needed at different points?

## Suggested hypotheses for a project

This is a big dataset! There are many potential hypotheses that you can explore using this dataset, or even by comparing it with other datasets.

- DNA replication machinery is periodically expressed in the cell cycle.
- Cell wall components are periodically expressed in the cell cycle.
- Functionally related genes are expressed at the same time as DNA replication machinery.
- Distinct cell-cycle promoting cyclins are expressed at different points of the cell cycle.
- Ribosomes are made at a distinct point in the cell cycle.
- Ribosomal biogenesis genes, that make ribosomal RNA, are made before ribosomal proteins are made.
- Gene expression patterns are essentially the same in the first and second cell cycle following arrest.
- Cell cycles proceed essentially the same way however we synchronise the cells, if we compare with datasets using other synchronisation methods.
- Cell cycle dependent gene expression patterns are evolutionarily conserved, if we compare with datasets from other species.

Some of these might use clustering methods such as those we have developed in the course.

A very brief [Overview of the Cell Cycle textbook chapter explains the need for DNA replication](https://www.ncbi.nlm.nih.gov/books/NBK26869/).

# Yeast gene name descriptions

To make more sense of this gene expression data, we need to know more about the genes we are measuring. Which of those are involved in DNA replication, for example? We have matched data containing information on protein-coding genes, including brief descriptions about what's known of gene function. These 2 datasets are linked by gene identifiers.

The file is `yeast_gene_name_descriptions_2023-09-11.tsv`. 
 
It was downloaded from the Saccharomyces Genome Database https://www.yeastgenome.org - this is a huge site that summarises everything known about this yeast. If you want to know more about some yeast genes, you can look up details there and link through to the primary research papers. 

In [ ]:
gene_df = pd.read_table("yeast_gene_name_descriptions_2023-09-11.tsv")

In [ ]:
gene_df.head()

There is a lot going on here. Each gene has a `primaryIdentifier`, a `secondaryIdentifier`, a `symbol`, and a `name`. Why do they need so many identifiers? One reason is that different databases containing different information need to connect to each other. Another reason is that there are different kinds of biological molecules - for example `YAL001C` is the name of the region of DNA that contains an open reading frame that encodes a protein sequence, and `TFC3`.

Let's look again at the gene names that came in our mRNA abundance data.

In [ ]:
df["gene"]

And compare to the gene name data

In [ ]:
gene_df["Gene.symbol"]

These look different, and even have different numbers of entries... oh no!?!

Reminder, this is a real-life example with published data derived from public databases and published papers. The problem of making one set of data correspond with another set of data from a different place is an everyday problem in real-life data. The tools we're working with in this course can help.

There are 2 important differences between the genes in these datasets, in fact:
1. The mRNA abundance dataset counts non-coding RNAs like `15S_rRNA` as well as messenger RNAs
2. The mRNA abundance dataset `gene` column is a mixture of protein names (in the `Gene.symbol` column of `gene_df`) and systematic names (in the `Gene.secondaryIdentifier` column).

If you want, you can check this yourself by looking through the lists and searching for some gene names on https://www.yeastgenome.org.

# Making the datasets talk to each other

There are different options as to how to make these agree. Here I will create a new column in `gene_df` called `gene` that contains the `Gene.symbol` if it exists and `Gene.secondaryIdentifier` otherwise.

In [ ]:
gene_df["gene"] = gene_df["Gene.symbol"].fillna(gene_df["Gene.secondaryIdentifier"])
gene_df

Check that this new column `gene` has the right contents: it's equal to `Gene.symbol` where that's defined, `Gene.secondaryIdentifier` otherwise.

Now we can merge the data frames.

In [ ]:
df_merged_all = gene_df.merge(right = df, on = "gene")

In [ ]:
df_merged_all

In [ ]:
df_merged_all.shape

Notice that this merged dataFrame has fewer rows than either precursor.

I'd guess that this is because some mRNAs weren't detected at all in the expression dataset. Or because the data were generated on a different genome release and the names have changed. It's not always possible to find out without analysing everything again from scratch, and there has to be a really good reason to do that

## Exclude dubious genes

Let's try focusing on genes where there is good evidence that a functional protein is produced. This is because an Open Reading Frame (ORF) is a predicted gene based on the sequence, but predictions are imperfect. As described in [New data and collaborations at the Saccharomyces Genome Database](https://doi.org/10.1093/genetics/iyab224):

> All *S. cerevisiae* ORFs are categorized into one of three groups: “verified” ORFs are those for which there is clear experimental evidence for the presence of expression of a protein-coding gene; “uncharacterized” ORFs are likely, but not yet established, to encode a protein; and “dubious” ORFs are unlikely to encode a protein.

If you look at `gene_df` you'll notice that the `Gene.qualifier` column contains values that correspond to these



In [ ]:
gene_df["Gene.qualifier"]

In [ ]:
gene_df["Gene.qualifier"].value_counts()

Let's remove the dubious values and see what we have left

In [ ]:
gene_df_nondubious = gene_df[ gene_df["Gene.qualifier"] != "Dubious" ]
gene_df_nondubious["Gene.qualifier"].value_counts()

In [ ]:
gene_df_nondubious.shape

Now let's merge the nondubious values

In [ ]:
df_merged = gene_df_nondubious.merge(right = df, on = "gene")

In [ ]:
df_merged.shape

Still less ... this is probably the best we can do for now.

This merged dataset on non-dubious genes could be a good place to start with connecting functional information in the `gene_df` to the mRNA abundance data. Or, for some analyses it might be easier to first choose a subset of the genes and then analyse the mRNA abundance data accordingly.

# Sanity-saving hint: decide how you want to load and clean up your data

This introduction has spent a lot of time loading, inspecting, tidying, and cleaning data.

You don't want to do that every time.

When you start your project work it's a good idea to:
1. decide what format you want the data in
2. figure out the minimal code that does these essential steps with your data
3. start a new notebook containing the minimal code and test it works

This is a sanity-saving approwch. You don't want to have one explosively messy document where you can't tell what's happening. You also don't want a hundred documents that take different approaches so you can't find anything. Try some things, then clean them up, and make a new and cleaner document when it's helpful.

## Picking cyclin-related genes out

As one example of how we could proceed to plot a subset of genes, let's find all the genes whose description involves "cyclin", then plot those.


In [ ]:
gene_df_cyclinrelated = gene_df[gene_df["Gene.description"].str.contains("cyclin")]
gene_df_cyclinrelated

Wait, that's missing CLN3. Let's search for `cyclin` and `Cyclin` etc, by ignoring the case of the search term.

Here I will use a regular expression (`re`) because that's the way I know how to search case-insensitively. I looked up help for [`pandas.Series.str.contains`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html).

In [ ]:
#importing regular expression module
import re

gene_df_cyclinrelated = gene_df[gene_df["Gene.description"].str.contains("cyclin", flags=re.IGNORECASE)]
gene_df_cyclinrelated

We'll use the same idea we used earlier for the CLN and CLB genes, search for  `df_tidy_CLNCLB_pivot` above

In [ ]:
df_tidy_cyclinrelated = df_tidy[ df_tidy.gene.isin( gene_df_cyclinrelated.gene )]
df_tidy_cyclinrelated_pivot = df_tidy_cyclinrelated.pivot(columns = "time_point", index = "gene")

sns.heatmap(data = np.log10(df_tidy_cyclinrelated_pivot), cmap = "bwr")

There's more work to do here to make an interpretable plot. You could try: 
- plotting log2 fold-change instead of log10(FPKM1).
- giving the axes helpful names and labels.
- adjusting plot size so every gene name appears.
- plotting fewer genes, after reading the descriptions to make sense of which to choose.
- clustering the plot so genes with similar abundance are plotted together.

Still, this is an example of how to explore the data.

# Bud index data

We have also shared, courtesy of Christina Kelliher, the data on number of budded cells in S. cerevisiae synchronized with alpha factor, that underlies Figure 1A. This gives you the opportunity to plot the percent budded cells, and compare to the transcriptional data, yourselves.

This is found in the excel file `2014-04-21_Kelliher_ScerevisiaeBudCounts.xlsx`, with column names:

- Time (min)
- 0 Bud
- 1 Bud
- Total
- % 1 bud

If you're reading this in to python, you'll need to start with `pandas.read_excel`, and fix the column names to be more python-friendly.

# Conclusion

Here we presented
- a dataset of yeast mRNA abundance during the cell cycle
- a dataset of yeast gene names and descriptions
- a few ideas about how to approach the data
- some potential hypotheses to address using the data

Your mission is to think through a hypothesis that you would like to address, then use the data provided and the tools taught in the course to address that hypothesis, and then to explain what you learned.

## References

For background reading, there is an excellent textbook on the cell cycle by David Morgan. In that, Section 3-12 "Transcriptional Control of Cell-Cycle Regulators" provides some background. If you are interested, other sections of the book give more information on many other aspects of the cell cycle. (You don't need to read this textbook in order to do the project.)

> The cell cycle : principles of control
> Morgan, David O.
> Northants : Oxford University Press; 2007, ©2007

This book is available [in the University library](https://discovered.ed.ac.uk/permalink/44UOE_INST/7g3mt6/alma9913098613502466), and as a [PDF on David Morgan's website](https://morganlab.ucsf.edu/sites/g/files/tkssra2561/f/wysiwyg/Morgan%20Cell%20Cycle%20Book.pdf).

David Morgan also has some [online lectures on the cell cycle at iBiology](https://www.ibiology.org/cell-biology/controlling-cell-cycle/).

Further references:

Comprehensive identification of cell cycle-regulated genes of the yeast Saccharomyces cerevisiae by microarray hybridization. Spellman, P.T. et al. Molecular Biology of the Cell, 1998. https://doi.org/10.1091/mbc.9.12.3273

Serial Regulation of Transcriptional Regulators in the Yeast Cell Cycle. Simon, I. et al. Cell, 2001. https://doi.org/10.1016/S0092-8674(01)00494-9

Investigating Conservation of the Cell-Cycle-Regulated Transcriptional Program in the Fungal Pathogen, Cryptococcus neoformans. Kelliher, C.M. et al. PLoS Genetics, 2016. https://doi.org/10.1371/journal.pgen.1006453

